In [ ]:
pip install python-docx

In [ ]:
pip install nltk

In [9]:
import pandas as pd
import re
import pathlib
import numpy as np
from IPython.display import display
from docx import Document
from pprint import pprint
from collections import defaultdict
pd.set_option('display.max_rows', 1000)

In [ ]:
def get_data():
  df = pd.read_pickle('./data/Formosan-Mandarin_sent_pairs_20220302.pkl')
  df_2 = df.astype(str, errors='ignore')
  df_2 = df_2.applymap(lambda x: x[1:] if x.startswith(".") else x)
  df_2 = df_2.applymap(lambda x: x.strip())
  filt = df_2.Ch.apply(len) < 5
  df_2 = df_2[~filt]
  return df, df_2

In [ ]:
data, data_2 = get_data()

In [ ]:
x = data[data['Lang_Ch'].str.contains("泰雅")]
x['Lang_Ch'].unique()

In [ ]:
data_2.shape

In [ ]:
r = data_2.copy()

def read_a_csv_and_drop_na(path):
    df = pd.read_csv(path, encoding="big5").dropna(how='all').dropna(axis=1, how='all')
    df['Lang_En'] = 'Atayal'
    df['Lang_Ch'] = '泰雅_賽考利克'
    df['From'] = "傳說故事精選篇"
    df = df.rename(columns={"Atayal": "Ab", "Mandarin": "Ch"})
    
    return df

for x in pathlib.Path("../atayal_data/").glob("*.csv"):
    c = read_a_csv_and_drop_na(x)
    r = pd.concat([r, c], ignore_index=True)
    


In [ ]:
r.columns
r.drop('Unnamed: 2', inplace=True,axis=1)

In [ ]:
r.to_pickle('./Formosan-Mandarin_sent_pairs_20220106.pkl', compression="gzip")

In [ ]:
df = pd.read_pickle('./Formosan-Mandarin_sent_pairs_20220106.pkl', compression="gzip")
df

# 02-16 更新:只要泰雅語就好, 將 Rau(1992) 資料加進來

In [ ]:
import pickle
import pandas as pd

df = pd.read_pickle('./data/Formosan-Mandarin_sent_pairs_20220106.pkl', compression="gzip")

df_atayal = df[df['Lang_En'] == 'Atayal']
print(df_atayal.shape)

r = df_atayal.copy()

In [ ]:
rau_1992 = pd.read_excel("data/Rau_1992_text_story.xlsx", sheet_name=None)

In [ ]:
rau_1992['01']

In [ ]:
for sheet in rau_1992:
    rau_1992[sheet] = rau_1992[sheet].dropna()
    print("shape of sheet", sheet, rau_1992[sheet].shape)
    rau_1992[sheet] = rau_1992[sheet].rename(columns={"Atayal": "Ab", "English": "Ch"})
    rau_1992[sheet]['Lang_En'] = 'Atayal'
    rau_1992[sheet]['Lang_Ch'] = '泰雅_賽考利克'
    rau_1992[sheet]['From'] = 'Rau (1992)'
    
    r = pd.concat([r, rau_1992[sheet]], ignore_index=True)
    print(r.shape)
    print('======')

In [ ]:
r.shape

In [ ]:
r.to_pickle('./data/Formosan-Mandarin_sent_pairs_20220216.pkl', compression="gzip")

# 02-17 更新: 整理大崁崁群的word檔

In [23]:
def 檢查各檔案的每個句子的tag(path):

    doc = Document(path)

    having_five_tags = False
    after_title_part = False
    after_new_line = False

    all_sentences = []

    for para in doc.paragraphs:
        
#         print(f'{para.text}\n###')
        
        ################################
        # BEGIN: 處理語料前面的metadata #       
        ################################
        
        if not after_title_part:
            if not para.text.startswith("A:") and\
            not para.text.startswith("RA:") and\
            not para.text.startswith("G:") and\
            not para.text.startswith("M:") and\
            not para.text.startswith("RM:"):
                continue

        after_title_part = True
        
        

        # 遇到換行
        if len(para.text.strip()) == 0:
            after_new_line = True
            all_sentences.append(dict())

        if not para.text.startswith("A:") and\
        not para.text.startswith("RA:") and\
        not para.text.startswith("G:") and\
        not para.text.startswith("M:") and\
        not para.text.startswith("RM:"):
            continue

        for p in para.text.split('\n'):
            if p.startswith("A:") or p.startswith("RA:") or p.startswith("G:") or p.startswith("M:") or p.startswith("RM:"):
                tag = p.split(":")[0]
                content = p.lstrip(tag + ":").strip()
                if content == 'none':
                    content = None

                if len(all_sentences) == 0:
                    all_sentences.append(dict())

                all_sentences[-1][tag] = content

    return all_sentences

In [ ]:
results = pd.DataFrame([{'Ab':None, 'Ch':None}])

for x in pathlib.Path("data/泰雅族大嵙崁群的部落故事").glob("*aligned.docx"):
    print(f"[[[{x}]]]")
    all_sentences = 檢查各檔案的每個句子的tag(x)
    all_sentences = pd.DataFrame(all_sentences)

    for index, row in all_sentences.iterrows():
        ab = ""
        ch = ""
        
        if 'RA' not in row:
            ab = row['A']
        else:
            if row['RA'] is None:
                ab = row['A']
            else:
                ab = row['RA']

        if 'RM' not in row:
            ch = row['M']
        else:
            if row['RM'] is None:
                ch = row['M']
            else:  
                ch = row['RM']
        
        if ab is None or ch is None:
            continue
            
        results = results.append({'Ab':ab, 'Ch':ch, 'Source': x.name[:4] + "_" + str(index)}, ignore_index = True)

In [ ]:
from IPython.display import display
pd.set_option('display.max_rows', 1000)

results = results.dropna()
results['From'] = '大嵙崁群故事'
results['Lang_En'] = 'Atayal'
results['Lang_Ch'] = '泰雅_賽考利克'

display(results.dropna())

In [ ]:
results[results['Source'] == 'TM01_3']

In [ ]:
import pickle
import pandas as pd

df = pd.read_pickle('./data/Formosan-Mandarin_sent_pairs_20220216.pkl', compression="gzip")

# df_atayal = df[df['Lang_En'] == 'Atayal']
# print(df_atayal.shape)

r = df.append(results, ignore_index=True)
r

In [ ]:
r.to_pickle('./data/Formosan-Mandarin_sent_pairs_20220217.pkl', compression="gzip")

# Huang 1994

In [ ]:
from docx import Document
from pprint import pprint

import pickle
import pandas as pd



In [ ]:
df = pd.read_pickle('./data/Formosan-Mandarin_sent_pairs_20220217.pkl', compression="gzip")
doc = Document("data/Huang_1994_text_aligned.docx")
    
results = []

for para in doc.paragraphs:
    print(para.text)
    
    if para.text.startswith("a:"):
        text = para.text.lstrip("a:").strip()
        results.append({"Ab": text})
    elif para.text.startswith("e:"):
        text = para.text.lstrip("e:").strip()
        results[-1]["Ch"] = text

results = pd.DataFrame(results)
results['From'] = 'Huang 1994'
results['Lang_En'] = 'Atayal'
results['Lang_Ch'] = '泰雅_賽考利克'

df = df.append(results, ignore_index=True)
df
df.to_pickle('./data/Formosan-Mandarin_sent_pairs_20220302.pkl', compression="gzip")

# 加進完整的大嵙崁

In [ ]:
results = pd.DataFrame([{'Ab':None, 'Ch':None, 'Source': None}])

for x in pathlib.Path("data/泰雅族大嵙崁群的部落故事").glob("*aligned.docx"):
    all_sentences = 檢查各檔案的每個句子的tag(x)
    all_sentences = pd.DataFrame(all_sentences)
#     display(all_sentences)
    
    for index, row in all_sentences.iterrows():
        ab = ""
        ch = ""
        
        try:
            ab = row['A'] if row['RA'] is None else row['RA']
            ch = row['M'] if row['RM'] is None else row['RM']
        except Exception as e:
            print(x)
            print(e)

        if ab is None or ch is None:
            continue
        
        df= pd.DataFrame({'Ab':[ab], 'Ch':[ch], 'Source': [f'{x.name[:4]}_{str(index)}']})
    
        results = pd.concat([results, df], ignore_index = True)

In [ ]:
from IPython.display import display
pd.set_option('display.max_rows', 1000)

results = results.dropna()
results['From'] = '大嵙崁群故事'
results['Lang_En'] = 'Atayal'
results['Lang_Ch'] = '泰雅_賽考利克'

display(results.dropna())


In [ ]:
df = pd.read_pickle('./data/Formosan-Mandarin_sent_pairs_20220302.pkl', compression="gzip")
df = pd.concat([df, results], ignore_index=True)
df.to_pickle('./data/Formosan-Mandarin_sent_pairs_20221102.pkl', compression="gzip")

# 加進復興鄉故事

In [ ]:
results = pd.DataFrame([{'Ab':None, 'Ch':None, 'Source': None}])

for x in pathlib.Path("data/復興鄉故事").glob("*aligned.docx"):
    all_sentences = 檢查各檔案的每個句子的tag(x)
    all_sentences = pd.DataFrame(all_sentences)
    all_sentences = all_sentences.replace({np.nan: None})
    display(all_sentences)
    
    for index, row in all_sentences.iterrows():
        ab = ""
        ch = ""
        
        try:
            ab = row['A'] if row['RA'] is None or len(row['RA']) == 0 else row['RA']
            ch = row['M'] if row['RM'] is None or len(row['RM']) == 0 else row['RM']
        except Exception as e:
            print(x)
            print(e)

        if ab is None or ch is None:
            continue
        
        df= pd.DataFrame({'Ab':[ab], 'Ch':[ch], 'Source': [f'{x.name[:4]}_{str(index)}']})
    
        results = pd.concat([results, df], ignore_index = True)

In [ ]:
results['From'] = '復興鄉故事'
results['Lang_En'] = 'Atayal'
results['Lang_Ch'] = '泰雅_賽考利克'

df = pd.read_pickle('./data/Formosan-Mandarin_sent_pairs_20221102.pkl', compression="gzip")
df = pd.concat([df, results], ignore_index=True)
df.to_pickle('./data/Formosan-Mandarin_sent_pairs_20221115.pkl', compression="gzip")

In [ ]:
df.to_pickle('./data/Formosan-Mandarin_sent_pairs_20221117.pkl', compression="gzip")

In [ ]:
list(df[df['Ab'].str.contains("none", na=False)]['Source'])

In [ ]:
for i, row in df.iterrows():

    if isinstance(row['Ab'], str) and 'G:' in row['Ab']:
        print(row['Ab'])
        print('@')

        if row['Ab'].startswith('none'):
            new = row['Ab'].replace('none\nG: ', '').strip()
        else:
            new = re.sub('G:.*','', row['Ab']).strip()
        df.at[i,'Ab'] = new
        print(new)
        print('----')


# 重新整理復興鄉

In [ ]:
df = pd.read_pickle('./data/Formosan-Mandarin_sent_pairs_20221117.pkl', compression="gzip")

In [36]:
df.shape

(12788, 6)

In [24]:
df = df[~df['From'].str.contains('復興鄉')]

In [35]:
df.shape

(12788, 6)

In [37]:
results = pd.DataFrame([{'Ab':None, 'Ch':None, 'Source': None}])

for x in pathlib.Path("data/復興鄉故事1").glob("*aligned.docx"):
    all_sentences = 檢查各檔案的每個句子的tag(x)
    all_sentences = pd.DataFrame(all_sentences)
    all_sentences = all_sentences.replace({np.nan: None})
#     display(all_sentences)
    
    for index, row in all_sentences.iterrows():
        ab = ""
        ch = ""
        
        try:
            ab = row['A'] if row['RA'] is None or len(row['RA']) == 0 else row['RA']
            ch = row['M'] if row['RM'] is None or len(row['RM']) == 0 else row['RM']
        except Exception as e:
            print(x)
            print(e)

        if ab is None or ch is None:
            continue
        
        dff= pd.DataFrame({'Ab':[ab], 'Ch':[ch], 'Source': [f'{x.name[:4]}_{str(index)}']})
    
        results = pd.concat([results, dff], ignore_index = True)

results['From'] = '復興鄉故事一'
results['Lang_En'] = 'Atayal'
results['Lang_Ch'] = '泰雅_賽考利克'

In [38]:
df = pd.concat([df, results], ignore_index=True)

In [39]:
df.shape

(13262, 6)

In [40]:
results = pd.DataFrame([{'Ab':None, 'Ch':None, 'Source': None}])

for x in pathlib.Path("data/復興鄉故事2").glob("*aligned.docx"):
    all_sentences = 檢查各檔案的每個句子的tag(x)
    all_sentences = pd.DataFrame(all_sentences)
    all_sentences = all_sentences.replace({np.nan: None})
#     display(all_sentences)
    
    for index, row in all_sentences.iterrows():
        ab = ""
        ch = ""
        
        try:
            ab = row['A'] if row['RA'] is None or len(row['RA']) == 0 else row['RA']
            ch = row['M'] if row['RM'] is None or len(row['RM']) == 0 else row['RM']
        except Exception as e:
            print(x)
            print(e)

        if ab is None or ch is None:
            continue
        
        dff= pd.DataFrame({'Ab':[ab], 'Ch':[ch], 'Source': [f'{x.name[:4]}_{str(index)}']})
    
        results = pd.concat([results, dff], ignore_index = True)
results = results.dropna()
        
display(results)
results['From'] = '復興鄉故事二'
results['Lang_En'] = 'Atayal'
results['Lang_Ch'] = '泰雅_賽考利克'

,Ab,Ch,Source
1,Hongu Utux,虹的故事,F201_0
2,"nanu qu , musa maku spqzyu misuw qani ga , zyw...",我現在所要講述的，是有關於虹的故事。,F201_1
3,"nanu squ raral , maki qutux qu bnkis , nkis nq...",從前，有一位老者，是部落的尊長，名字叫布達‧拿威，是一位非常好的人。,F201_2
4,"hizya qani qu , syan inlungan na qalang , baha...",部落裏的人都非常尊敬他，因為他會照顧人，也會教導人，從不歧視人，是他們所敬仰的人。他們都很聽...,F201_3
5,"ana ga , babaw , babaw nya lga , mnkis uzi qu ...",雖然如此，隨著歲月的流逝，他也慢慢變老了。,F201_4
6,"nanu yasa qu , baqun nya muci ma iyat kzinga m...",因此，他心裹明白，即將不久於人世，他就召集所有部落的人，並告訴他們：,F201_5
7,"babaw na huqil saku ga , Utux maku ga , musa s...",「我死了之後，我的靈將會一直在天上，我的靈看超來是紅色的，你們會天天看到，我將會在那裏照顧你...,F201_6
8,"nanu yasa qu , ini lawa bsyaq lga, ktan nha ma...",在他死了之後沒有多久，在山的那一邊，部落的水源處看見虹，非常的好看，像是一座橋，架於兩座山之...,F201_7
9,"mtrang mhtuw qu hongu na Utux lga , msbisuw ru...",當虹出現的時候，天際會打雷、出現閃電，照亮天際，聽到隆隆的巨響，不是嗎？,F201_8
10,"hnswa nya muci ga , cyux mqwas qu Buta Naway s...",他們都認為就是因為布達‧拿威在唱歌。這種說法是我去拜訪親友時他告訴我的。,F201_9


In [41]:
df = pd.concat([df, results], ignore_index=True)

In [43]:
df.to_pickle('./data/Formosan-Mandarin_sent_pairs_20221226.pkl', compression="gzip")

# Egerod 1974 & Huang 1993 & Rau et al. 1995

In [ ]:
df = pd.read_pickle('./data/Formosan-Mandarin_sent_pairs_20221207.pkl', compression="gzip")

    
results = []

doc = Document("data/Egerod_1974_text.docx")

for para in doc.paragraphs:
    if para.text.startswith("a:"):
        text = para.text.lstrip("a:").strip()
        results.append({"Ab": text})
    elif para.text.startswith("e:"):
        text = para.text.lstrip("e:").strip()
        results[-1]["Ch"] = text   
        
results = pd.DataFrame(results)
results['From'] = 'Egerod (1974)'
results['Lang_En'] = 'Atayal'
results['Lang_Ch'] = '泰雅_賽考利克'

df = pd.concat([df, results], ignore_index=True)

# df.to_pickle('./data/Formosan-Mandarin_sent_pairs_20221219.pkl', compression="gzip")

In [ ]:
results = []

doc = Document("data/Huang_1993_text.docx")

for para in doc.paragraphs:
    if para.text.startswith("a:"):
        text = para.text.lstrip("a:").strip()
        results.append({"Ab": text})
    elif para.text.startswith("e:"):
        text = para.text.lstrip("e:").strip()
        results[-1]["Ch"] = text   
        
results = pd.DataFrame(results)
results['From'] = 'Huang (1993)'
results['Lang_En'] = 'Atayal'
results['Lang_Ch'] = '泰雅_賽考利克'

df = pd.concat([df, results], ignore_index=True)

# df.to_pickle('./data/Formosan-Mandarin_sent_pairs_20221219.pkl', compression="gzip")

In [ ]:
results = []

doc = Document("data/Rau_1995_text.docx")

for para in doc.paragraphs:
    if para.text.startswith("a:"):
        text = para.text.lstrip("a:").strip()
        results.append({"Ab": text})
    elif para.text.startswith("m:"):
        text = para.text.lstrip("m:").strip()
        results[-1]["Ch"] = text   
        
results = pd.DataFrame(results)
results['From'] = 'Rau et al. (1995)'
results['Lang_En'] = 'Atayal'
results['Lang_Ch'] = '泰雅_賽考利克'

df = pd.concat([df, results], ignore_index=True)

# df.to_pickle('./data/Formosan-Mandarin_sent_pairs_20221219.pkl', compression="gzip")

In [ ]:
df.to_pickle('./data/Formosan-Mandarin_sent_pairs_20221219.pkl', compression="gzip")

# 統計各文本字數

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize

def 生成語料描述統計pickle檔(corpus_pickel_path):
    df = pd.read_pickle(corpus_pickel_path, compression="gzip")
    df['總詞數 (族語)'] = df['Ab'].str.split(' ').str.len()
    df['總句數 (族語)'] = df['Ab'].astype('str').apply(lambda s: len(sent_tokenize(s)))
    df['總句數 (族語)'] = df['總句數 (族語)'].astype('float64')
    summarized_df = df.groupby(['From']).sum()
    summarized_df['平均每句詞數 (族語)'] = summarized_df['總詞數 (族語)'] / summarized_df['總句數 (族語)']
    summarized_df.to_pickle('./data/basic_stats.pkl', compression="gzip")
    return summarized_df

In [ ]:
a = 生成語料描述統計pickle檔('./data/Formosan-Mandarin_sent_pairs_20221219.pkl')

In [ ]:
a

In [ ]:
pd.read_pickle('./data/basic_stats.pkl', compression="gzip")

# Huang and Wu 2016

In [ ]:
df = pd.read_pickle('./data/Formosan-Mandarin_sent_pairs_20221219.pkl', compression="gzip")

    
results = []

doc = Document("data/Huang&Wu_text_1_aligned.docx")
i = 0

for para in doc.paragraphs:
    if para.text.startswith("a:"):
        text = para.text.lstrip("a:").strip().replace('\xa0', ' ')
        results.append({"Ab": text, "Source": f"1_{i}"})
        i += 1
    elif para.text.startswith("m:"):
        text = para.text.lstrip("m:").strip()
        results[-1]["Ch"] = text   

results = pd.DataFrame(results)
results['From'] = 'Huang & Wu (2016)'
results['Lang_En'] = 'Atayal'
results['Lang_Ch'] = '泰雅_賽考利克'


df = pd.concat([df, results], ignore_index=True)

# df.to_pickle('./data/Formosan-Mandarin_sent_pairs_20221219.pkl', compression="gzip")

In [26]:
df = pd.read_pickle('./data/Formosan-Mandarin_sent_pairs_20221226.pkl', compression="gzip")


In [22]:
a = 檢查各檔案的每個句子的tag('E:/clone/Formosan-languages/data/復興鄉故事2/F210_aligned.docx')
a

[{'A': 'Kneril ms’ari’',
  'RA': 'Kneril Ms’aris ?',
  'G': 'Kneril Ms’aris',
  'M': '貪心婦人的故事',
  'RM': None},
 {'A': 'kmyala saku qotux musa smatu sqoliq ga , musa smatu sqoliq ,',
  'RA': 'kmyala saku qutux musa smatu squliq ga , musa smatu squliq ,',
  'G': 'kmyala saku qutux musa smatu squliq ga , musa smatu squliq ,',
  'M': '我說一則有關於貪心婦人在喪禮的故事。',
  'RM': ''},
 {'A': 'a ~ maki qotux kneril yaki qasa . rro , ini pijang qnjats , lokah lokah mtjwaw hiya ga , ungats ana qotux picywagan nya qmayah nya hiya nanak . si giway musa pgluw pqabu sqoliq nanu ,',
  'RA': 'a ~ maki qutux kneril yaki qasa . rro , ini pzyang qnzyat , lokah lokah mtzywaw hiya ga , ungat ana qutux pincywagan nya qmayah nya hiya nanak . si giway musa pgluw pqabu squliq nanu ,',
  'G': 'a ~ maki qutux kneril yaki qasa . rro , ini pzyang qnzyat , lokah lokah mtzywaw hiya ga , ungat ana qutux pincywagan nya qmayah nya hiya nanak . si giway musa pgluw pqabu squliq nanu ,',
  'M': '這位婦人不怎麼辛勤工作。雖然有在工作，卻看不到山上工作的成果，就只好一直被人雇工

In [32]:
df = df[df['From'] != '復興鄉故事二']

In [33]:
df.From.unique()

array(['詞典', '文法', '句型', '生活會話', '九階教材', '傳說故事精選篇', 'Rau (1992)',
       'Huang (1994)', '大嵙崁群故事', 'Egerod (1974)', 'Huang (1993)',
       'Rau et al. (1995)'], dtype=object)

In [24]:
df = pd.read_pickle('./data/Formosan-Mandarin_sent_pairs_20221226.pkl', compression="gzip")
df.to_pickle('./data/Formosan-Mandarin_sent_pairs_20221226-2.pkl', compression="gzip")

In [23]:
x=df[df['From']=='復興鄉故事二']
zz = x[x['Ab'].str.contains("mkayal ga")]['Ab']
pd.options.display.max_colwidth = 1000
zz.to_string()
display(zz)
# type(zz)

13490    nanu maki qutux ryax , musa mluw smatu squliq , musa mluw smatu squliq qasa lga , nanu a ~ ini nha , ini nha szyay izyal maras lga , hiya lga musa sayux lpyung muci mkayal ga ,
Name: Ab, dtype: object